# LangChain Pipeline 核心技術

歡迎來到 LangChain Pipeline 核心技術課程！本課程將深入介紹 LangChain 管道設計的關鍵技術。

## 🎯 課程目標

在第 03 課中，我們學會了 PromptTemplate 的進階應用。現在我們要學習如何將這些組件組合成強大的處理管道。

### 📋 本課程涵蓋的關鍵技術

1. **LangChain 管道（Pipeline）** - 使用 `|` 運算子組合組件
2. **結構化輸出與 JSON 解析** - 讓 LLM 返回結構化數據
3. **錯誤處理與容錯機制** - 生產環境的穩定性保證
4. **批量處理與 API 限制處理** - 大規模數據分析準備
5. **輸出解析器（Output Parsers）** - 自動化結果處理
6. **管道監控與調試** - 確保管道穩定運行

讓我們開始深入學習 LangChain Pipeline 的核心技術！

## 📦 環境設定與依賴安裝

首先確保我們有完整的 LangChain 生態系統。

In [ ]:
# 安裝完整的 LangChain 生態系統
!pip install langchain langchain-google-genai langchain-community langchain-core python-dotenv pandas plotly

In [1]:
# 導入所有必要的模組
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from pydantic import BaseModel, Field  # 更新為 Pydantic v2
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
import json
import time
import re
from typing import List, Dict, Any

import os
from dotenv import load_dotenv

load_dotenv()

llm = ChatGoogleGenerativeAI(
    model=os.getenv("MODEL", "gemini-1.5-flash"),
    temperature=os.getenv("TEMPERATURE", 0.7),
    google_api_key=os.getenv("API_KEY"),
)

## 🎯 準備 PromptTemplate

我們將使用第 03 課學到的 PromptTemplate 技巧來建立管道。

In [2]:
# 準備基本的 PromptTemplate（來自第 03 課）
basic_prompt = PromptTemplate(
    input_variables=["question"],
    template="請回答以下問題：{question}"
)

# 飲料品牌識別 Prompt（來自第 03 課）
brand_identification_prompt = PromptTemplate(
    input_variables=["text"],
    template="""
你是一位專業的台灣飲料市場分析師。請分析以下文本，識別其中提到的飲料品牌。

文本：{text}

請以 JSON 格式返回結果：
{{
    "brands": ["品牌1", "品牌2", ...],
    "confidence": "高/中/低",
    "reasoning": "識別理由"
}}

注意：
- 只識別台灣常見的飲料品牌（如：CoCo、50嵐、迷客夏、清心福全等）
- 如果沒有明確品牌，brands 返回空陣列
- 請判斷識別的信心程度
"""
)

print("✅ PromptTemplate 準備完成")
print("🎯 現在開始學習如何組合成管道")

✅ PromptTemplate 準備完成
🎯 現在開始學習如何組合成管道


## 🔗 LangChain 管道（Pipeline）核心技術

LangChain 的管道系統是其最強大的特性之一。使用 `|` 運算子可以輕鬆組合不同的組件。

### 🎯 管道設計概念

管道（Pipeline）是 LangChain 的核心概念：
- **模組化設計**: 每個組件都有明確的職責
- **鏈式組合**: 使用 `|` 運算子串接組件
- **數據流動**: 數據從一個組件流向下一個組件
- **錯誤傳播**: 錯誤會在管道中傳播和處理

讓我們從簡單的管道開始學習！

In [5]:
# 1. 基本管道：Prompt + LLM
print("🔗 建立基本管道：PromptTemplate | LLM")

# 建立簡單的問答管道
qa_chain = basic_prompt | llm

print("✅ 基本管道已建立")
print("🎯 管道組成：PromptTemplate → LLM")
print("\n測試基本管道：")

try:
    result = qa_chain.invoke({"question": "請簡短介紹 LangChain"})
    print(f"📤 輸入：請簡短介紹 LangChain")
    print(f"📥 輸出：{result.content}")
except Exception as e:
    print(f"❌ 錯誤：{e}")

print("\n" + "="*50)

🔗 建立基本管道：PromptTemplate | LLM
✅ 基本管道已建立
🎯 管道組成：PromptTemplate → LLM

測試基本管道：
📤 輸入：請簡短介紹 LangChain
📥 輸出：LangChain 是一個用於開發基於大型語言模型 (LLM) 的應用程式的框架。 簡單來說，它提供了一組工具、組件和介面，讓你可以更方便地：

* **連接 LLM 與外部數據源：** 像是資料庫、API、文件等等，讓 LLM 能夠獲取更多資訊。
* **構建複雜的 LLM 應用流程：** 例如問答系統、聊天機器人、文件摘要等等，可以將多個 LLM 操作串聯起來。
* **提高 LLM 應用程式的可解釋性和可控性：** 方便追蹤 LLM 的推理過程，並進行干預和調整。

總之，LangChain 讓開發者更容易利用 LLM 的強大能力，打造更智能、更實用的應用程式。



In [5]:
# 2. 進階管道：Prompt + LLM + Output Parser
print("🔗 建立進階管道：PromptTemplate | LLM | OutputParser")

# 字符串輸出解析器
str_parser = StrOutputParser()

# 建立進階管道
advanced_chain = basic_prompt | llm | str_parser

print("✅ 進階管道已建立")
print("🎯 管道組成：PromptTemplate → LLM → StrOutputParser")
print("\n測試進階管道：")

try:
    result = advanced_chain.invoke({"question": "簡短描述什麼是機器學習？"})
    print(f"📤 輸入：簡短描述什麼是機器學習？")
    print(f"📥 輸出類型：{type(result)}")
    print(f"📥 輸出內容：{result[:100]}...")
except Exception as e:
    print(f"❌ 錯誤：{e}")

print("\n" + "="*50)

🔗 建立進階管道：PromptTemplate | LLM | OutputParser
✅ 進階管道已建立
🎯 管道組成：PromptTemplate → LLM → StrOutputParser

測試進階管道：
📤 輸入：簡短描述什麼是機器學習？
📥 輸出類型：<class 'str'>
📥 輸出內容：機器學習是一種讓電腦在**沒有明確程式指令**的情況下，**從數據中學習**，並**自動改進**其性能的技術。 簡單來說，就是讓電腦自己找規律，並利用這些規律來做出預測或決策。...



### 🤔 為什麼需要 StrOutputParser？

很多人會問：**「LLM API 回傳的不是已經是字串了嗎？為什麼還需要 StrOutputParser？」**

這是一個很棒的問題！讓我們來看看實際的情況：

#### 📋 LLM 回傳的實際格式

```python
# 直接調用 LLM 時，返回的是 AIMessage 物件
result = llm.invoke("你好")
print(type(result))        # <class 'langchain_core.messages.ai.AIMessage'>
print(result.content)      # 這才是我們要的字串內容
```

#### 🎯 StrOutputParser 的作用

1. **物件轉字串**：將 `AIMessage` 物件轉換為純字串
2. **管道統一性**：確保管道中的數據格式一致
3. **後續處理**：便於進行進一步的字串操作
4. **型別安全**：明確數據類型，避免混淆

#### 💡 實際比較

| 方法 | 回傳類型 | 取得內容方式 | 管道友善度 |
|------|----------|--------------|------------|
| `llm.invoke()` | `AIMessage` | `result.content` | ❌ 需要額外處理 |
| `llm \| StrOutputParser()` | `str` | 直接使用 | ✅ 完美整合 |

#### 🔗 在 Pipeline 中的重要性

```python
# ❌ 沒有 StrOutputParser - 需要手動處理
chain = prompt | llm
result = chain.invoke({"question": "..."})
text = result.content  # 需要額外步驟

# ✅ 有 StrOutputParser - 直接獲得字串
chain = prompt | llm | StrOutputParser()
text = chain.invoke({"question": "..."})  # 直接是字串
```

**結論**：`StrOutputParser` 讓管道更加簡潔和一致！

In [6]:
# 🔧 進階範例：StrOutputParser 在複雜管道中的重要性
print("🔧 複雜管道中的 StrOutputParser 重要性")
print("=" * 50)

# 假設我們要建立一個：問答 → 文字清理 → 長度統計 的管道

def text_cleaner(text):
    """文字清理函數：移除多餘空格和標點"""
    if hasattr(text, 'content'):  # 如果是 AIMessage 物件
        text = text.content
    # 清理文字
    cleaned = re.sub(r'[^\w\s\u4e00-\u9fff]', '', text)
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    return cleaned

def text_stats(text):
    """統計文字長度"""
    return {
        'original': text,
        'length': len(text),
        'words': len(text.split()),
        'chars_no_space': len(text.replace(' ', ''))
    }

print()

# ✅ 有 StrOutputParser 的管道
print("✅ 有 StrOutputParser 的管道：")
try:
    working_chain = (
        basic_prompt | 
        llm |
        StrOutputParser() |  # 將 AIMessage 轉為字串
        RunnableLambda(text_cleaner) |
        RunnableLambda(text_stats)
    )
    
    result = working_chain.invoke({"question": "簡短描述什麼是程式設計？"})
    print(f"✅ 成功執行！")
    print(f"📊 統計結果：")
    print(f"   原文長度：{result['length']} 字符")
    print(f"   單詞數量：{result['words']} 個")
    print(f"   無空格字符：{result['chars_no_space']} 個")
    print(f"   清理後內容：{result['original'][:50]}...")
    
except Exception as e:
    print(f"❌ 錯誤：{e}")

print()
print("   💥 沒有 StrOutputParser：後續函數會收到錯誤的數據類型")
print("   ✅ 有 StrOutputParser：數據類型統一，管道順暢運行")
print("   📏 這在建立複雜管道時特別重要！")

🔧 複雜管道中的 StrOutputParser 重要性

✅ 有 StrOutputParser 的管道：
✅ 成功執行！
📊 統計結果：
   原文長度：50 字符
   單詞數量：1 個
   無空格字符：50 個
   清理後內容：程式設計是指使用特定的程式語言撰寫指令讓電腦執行特定任務或解決問題的過程簡單來說就是告訴電腦要做什麼...

   💥 沒有 StrOutputParser：後續函數會收到錯誤的數據類型
   ✅ 有 StrOutputParser：數據類型統一，管道順暢運行
   📏 這在建立複雜管道時特別重要！


In [8]:
# 3. 實戰管道：品牌識別管道
print("🏪 建立品牌識別管道")

# 建立品牌識別鏈
brand_identification_chain = brand_identification_prompt | llm | str_parser

print("✅ 品牌識別管道已建立")
print("🎯 管道組成：品牌識別Prompt → LLM → 字符串解析器")
print("\n測試品牌識別管道：")

test_text = "昨天去 CoCo 買了焦糖奶茶，今天想試試看 50嵐 的波霸奶茶。朋友推薦迷客夏的芝芝系列也很棒。"

try:
    result = brand_identification_chain.invoke({"text": test_text})
    print(f"📤 輸入文本：{test_text}")
    print(f"📥 LLM 原始輸出：{result}")
    
    # 嘗試解析 JSON
    try:
        # 清理可能的代碼區塊包裝
        cleaned_result = result.strip()
        if cleaned_result.startswith('```json'):
            cleaned_result = cleaned_result.replace('```json', '').replace('```', '').strip()
        
        parsed = json.loads(cleaned_result)
        print(f"\n✅ JSON 解析成功：")
        print(f"🏪 識別的品牌：{parsed.get('brands', [])}")
        print(f"📊 信心程度：{parsed.get('confidence', 'N/A')}")
        print(f"🧠 識別理由：{parsed.get('reasoning', 'N/A')}")
    except json.JSONDecodeError:
        print("⚠️ JSON 解析失敗，這就是為什麼我們需要健壯的錯誤處理！")
        
except Exception as e:
    print(f"❌ 管道執行錯誤：{e}")

🏪 建立品牌識別管道
✅ 品牌識別管道已建立
🎯 管道組成：品牌識別Prompt → LLM → 字符串解析器

測試品牌識別管道：
📤 輸入文本：昨天去 CoCo 買了焦糖奶茶，今天想試試看 50嵐 的波霸奶茶。朋友推薦迷客夏的芝芝系列也很棒。
📥 LLM 原始輸出：```json
{
    "brands": ["CoCo", "50嵐", "迷客夏"],
    "confidence": "高",
    "reasoning": "文本中明確提及CoCo、50嵐、迷客夏三個台灣常見的飲料品牌。"
}
```

✅ JSON 解析成功：
🏪 識別的品牌：['CoCo', '50嵐', '迷客夏']
📊 信心程度：高
🧠 識別理由：文本中明確提及CoCo、50嵐、迷客夏三個台灣常見的飲料品牌。


## 📊 結構化輸出與 JSON 解析

在實際應用中，我們需要 LLM 返回結構化的數據。

### 🎯 為什麼需要結構化輸出？

1. **程式化處理**: 結構化數據便於程式處理
2. **數據驗證**: 可以驗證輸出格式的正確性
3. **批量分析**: 支援大規模數據分析
4. **結果統計**: 便於進行統計和視覺化

讓我們學習如何實現穩定的結構化輸出！

In [9]:
# 1. 定義輸出結構
from pydantic import BaseModel, Field
from typing import List

class BrandAnalysis(BaseModel):
    """品牌分析結果的結構定義"""
    brands: List[str] = Field(description="識別出的飲料品牌列表")
    confidence: str = Field(description="識別信心程度：高/中/低")
    reasoning: str = Field(description="識別理由")

print("📋 定義了 BrandAnalysis 結構")
print("✅ 這將確保 LLM 輸出符合我們的需求")

# 展示結構
print("\n🏗️ 輸出結構：")
print(f"• brands: {BrandAnalysis.model_fields['brands'].description}")
print(f"• confidence: {BrandAnalysis.model_fields['confidence'].description}")
print(f"• reasoning: {BrandAnalysis.model_fields['reasoning'].description}")

📋 定義了 BrandAnalysis 結構
✅ 這將確保 LLM 輸出符合我們的需求

🏗️ 輸出結構：
• brands: 識別出的飲料品牌列表
• confidence: 識別信心程度：高/中/低
• reasoning: 識別理由


In [10]:
# 2. 使用 JsonOutputParser
from langchain_core.output_parsers import JsonOutputParser

# 建立 JSON 解析器
json_parser = JsonOutputParser(pydantic_object=BrandAnalysis)

print("🔧 建立 JSON 輸出解析器")
print("📋 獲取格式說明：")
print(json_parser.get_format_instructions())

🔧 建立 JSON 輸出解析器
📋 獲取格式說明：
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "品牌分析結果的結構定義", "properties": {"brands": {"description": "識別出的飲料品牌列表", "items": {"type": "string"}, "title": "Brands", "type": "array"}, "confidence": {"description": "識別信心程度：高/中/低", "title": "Confidence", "type": "string"}, "reasoning": {"description": "識別理由", "title": "Reasoning", "type": "string"}}, "required": ["brands", "confidence", "reasoning"]}
```


In [11]:
# 3. 改進的品牌識別 Prompt（包含格式說明）
structured_brand_prompt = PromptTemplate(
    input_variables=["text"],
    template="""
你是一位專業的台灣飲料市場分析師。請分析以下文本，識別其中提到的飲料品牌。

文本：{text}

{format_instructions}

注意：
- 只識別台灣常見的飲料品牌（如：CoCo、50嵐、迷客夏、清心福全等）
- 如果沒有明確品牌，brands 返回空陣列
- confidence 請填入：高/中/低
""",
    partial_variables={"format_instructions": json_parser.get_format_instructions()}
)

print("✅ 改進的結構化 Prompt 已定義")
print("🎯 包含了自動生成的格式說明")

✅ 改進的結構化 Prompt 已定義
🎯 包含了自動生成的格式說明


In [12]:
# 4. 建立結構化輸出管道
structured_brand_chain = structured_brand_prompt | llm | json_parser

print("🔗 建立結構化品牌分析管道")
print("🎯 管道組成：結構化Prompt → LLM → JSON解析器")

# 測試結構化輸出
test_text = "今天心情不錯，先去 CoCo 買了珍珠奶茶，下午又去 50嵐 買了波霸奶茶。晚上朋友推薦去迷客夏試試看芝芝系列。"

print(f"\n📤 測試文本：{test_text}")

try:
    result = structured_brand_chain.invoke({"text": test_text})
    print("\n✅ 結構化輸出成功：")
    print(f"📊 輸出類型：{type(result)}")
    print(f"🏪 識別品牌：{result.get('brands', [])}")
    print(f"📈 信心程度：{result.get('confidence', 'N/A')}")
    print(f"🧠 分析理由：{result.get('reasoning', 'N/A')}")
    
except Exception as e:
    print(f"❌ 結構化輸出失敗：{e}")
    print("💡 這就是為什麼我們需要容錯機制！")

🔗 建立結構化品牌分析管道
🎯 管道組成：結構化Prompt → LLM → JSON解析器

📤 測試文本：今天心情不錯，先去 CoCo 買了珍珠奶茶，下午又去 50嵐 買了波霸奶茶。晚上朋友推薦去迷客夏試試看芝芝系列。

✅ 結構化輸出成功：
📊 輸出類型：<class 'dict'>
🏪 識別品牌：['CoCo', '50嵐', '迷客夏']
📈 信心程度：高
🧠 分析理由：文本中明確提及CoCo、50嵐和迷客夏，皆為台灣常見且知名的連鎖飲料品牌。


## 🛡️ 錯誤處理與容錯機制

在生產環境中，穩定性至關重要。讓我們學習如何建立健壯的錯誤處理機制。

In [13]:
# 1. 基本錯誤處理
def safe_brand_analysis(text: str, max_retries: int = 3) -> Dict[str, Any]:
    """安全的品牌分析函數，包含重試機制"""
    
    for attempt in range(max_retries):
        try:
            print(f"🔄 嘗試分析（第 {attempt + 1} 次）...")
            
            # 使用結構化管道
            result = structured_brand_chain.invoke({"text": text})
            
            # 驗證結果
            if isinstance(result, dict) and 'brands' in result:
                print("✅ 結構化分析成功")
                return result
            else:
                raise ValueError("輸出格式不正確")
                
        except json.JSONDecodeError as e:
            print(f"⚠️ JSON 解析失敗（第 {attempt + 1} 次）：{e}")
            if attempt == max_retries - 1:
                return _fallback_analysis(text)
            
        except Exception as e:
            print(f"❌ 分析失敗（第 {attempt + 1} 次）：{e}")
            if attempt == max_retries - 1:
                return _fallback_analysis(text)
            
        # 重試前等待
        if attempt < max_retries - 1:
            time.sleep(1)
    
    return _fallback_analysis(text)

def _fallback_analysis(text: str) -> Dict[str, Any]:
    """備用分析方法：使用簡單的關鍵字匹配"""
    print("🔄 使用備用分析方法...")
    
    # 常見台灣飲料品牌關鍵字
    brand_keywords = [
        'CoCo', '50嵐', '迷客夏', '清心福全', '麥吉', 'Comebuy', 
        '茶湯會', '一芳', '鮮茶道', '大苑子', 'Mr.Wish'
    ]
    
    found_brands = []
    for brand in brand_keywords:
        if brand in text:
            found_brands.append(brand)
    
    return {
        'brands': found_brands,
        'confidence': '低',
        'reasoning': '使用備用關鍵字匹配方法'
    }

print("🛡️ 錯誤處理機制已定義")
print("✅ 包含重試機制和備用分析方法")

🛡️ 錯誤處理機制已定義
✅ 包含重試機制和備用分析方法


In [15]:
# 2. 測試錯誤處理機制
print("🧪 測試錯誤處理機制")

test_cases = [
    "昨天去 CoCo 買了珍珠奶茶，很好喝！",
    "推薦 50嵐 的波霸奶茶和迷客夏的芝芝系列",
    "今天天氣很好，但沒有提到任何飲料品牌"
]

for i, test_text in enumerate(test_cases, 1):
    print(f"\n📋 測試案例 {i}：{test_text}")
    print("-" * 40)
    
    result = safe_brand_analysis(test_text)
    
    print(f"🏪 識別品牌：{result['brands']}")
    print(f"📊 信心程度：{result['confidence']}")
    print(f"🧠 分析理由：{result['reasoning']}")

print("\n🎯 這種健壯的錯誤處理將確保穩定性！")

🧪 測試錯誤處理機制

📋 測試案例 1：昨天去 CoCo 買了珍珠奶茶，很好喝！
----------------------------------------
🔄 嘗試分析（第 1 次）...
✅ 結構化分析成功
🏪 識別品牌：['CoCo']
📊 信心程度：高
🧠 分析理由：文本中明確提及 'CoCo'，這是在台灣廣為人知的飲料品牌名稱。

📋 測試案例 2：推薦 50嵐 的波霸奶茶和迷客夏的芝芝系列
----------------------------------------
🔄 嘗試分析（第 1 次）...
✅ 結構化分析成功
🏪 識別品牌：['50嵐', '迷客夏']
📊 信心程度：高
🧠 分析理由：文本中明確提及了'50嵐'和'迷客夏'這兩個台灣常見的飲料品牌。

📋 測試案例 3：今天天氣很好，但沒有提到任何飲料品牌
----------------------------------------
🔄 嘗試分析（第 1 次）...
✅ 結構化分析成功
🏪 識別品牌：[]
📊 信心程度：高
🧠 分析理由：文本中沒有提及任何飲料品牌。

🎯 這種健壯的錯誤處理將確保穩定性！


## ⚡ 批量處理與 API 限制處理

在處理大量數據時，我們需要考慮 API 限制和效率。

### 🎯 批量處理的重要性

1. **效率提升**: 批量處理減少開銷
2. **API 限制**: 避免超過 API 調用限制
3. **錯誤恢復**: 單個失敗不影響整體
4. **進度追蹤**: 實時監控處理進度
5. **資源管理**: 合理使用系統資源

讓我們學習如何設計健壯的批量處理系統！

In [18]:
# 1. 批量處理函數
def batch_brand_analysis(texts: List[str], batch_size: int = 5, delay: float = 1.0) -> List[Dict[str, Any]]:
    """批量品牌分析，包含速率限制"""
    
    results = []
    total = len(texts)
    
    print(f"🚀 開始批量分析 {total} 個文本")
    print(f"⚙️ 批次大小：{batch_size}，延遲：{delay}秒")
    
    for i in range(0, total, batch_size):
        batch = texts[i:i + batch_size]
        batch_num = i // batch_size + 1
        
        print(f"\n📦 處理批次 {batch_num}/{(total - 1) // batch_size + 1}")
        
        for j, text in enumerate(batch):
            text_num = i + j + 1
            print(f"  📝 分析文本 {text_num}/{total}: {text[:30]}...")
            
            try:
                result = safe_brand_analysis(text, max_retries=2)
                result['text_id'] = text_num
                result['original_text'] = text
                results.append(result)
                
                print(f"  ✅ 完成：{result['brands']}")
                
            except Exception as e:
                print(f"  ❌ 失敗：{e}")
                results.append({
                    'text_id': text_num,
                    'original_text': text,
                    'brands': [],
                    'confidence': '無',
                    'reasoning': f'處理失敗：{str(e)}'
                })
            
            # API 限制延遲
            if j < len(batch) - 1:  # 不是批次的最後一個
                time.sleep(delay)
        
        # 批次間延遲
        if i + batch_size < total:
            print(f"  ⏸️ 批次完成，等待 {delay * 2} 秒...")
            time.sleep(delay * 2)
    
    print(f"\n🎉 批量分析完成！成功處理 {len(results)} 個文本")
    return results

print("⚡ 批量處理機制已定義")
print("✅ 包含速率限制和錯誤處理")

⚡ 批量處理機制已定義
✅ 包含速率限制和錯誤處理


In [26]:
# 2. 測試批量處理
print("🧪 測試批量處理")

sample_texts = [
    "今天去 CoCo 買珍珠奶茶",
    "50嵐 的波霸奶茶很棒",
    "迷客夏的芝芝系列推薦",
    "清心福全的檸檬綠茶",
    "今天天氣很好但沒喝飲料",
    "麥吉的氣泡飲料很特別"
]

# 執行批量分析（較小的延遲用於演示）
batch_results = batch_brand_analysis(sample_texts, batch_size=3, delay=0.5)

# 分析結果
print("\n📊 批量分析結果統計：")
total_brands = []
for result in batch_results:
    total_brands.extend(result['brands'])

from collections import Counter
brand_counts = Counter(total_brands)

print(f"🏪 總共識別出 {len(set(total_brands))} 個不同品牌")
print(f"📈 品牌統計：{dict(brand_counts)}")
print(f"✅ 成功率：{len([r for r in batch_results if r['brands']]) / len(batch_results) * 100:.1f}%")

🧪 測試批量處理
🚀 開始批量分析 6 個文本
⚙️ 批次大小：3，延遲：0.5秒

📦 處理批次 1/2
  📝 分析文本 1/6: 今天去 CoCo 買珍珠奶茶...
🔄 嘗試分析（第 1 次）...
✅ 結構化分析成功
  ✅ 完成：['CoCo']
  📝 分析文本 2/6: 50嵐 的波霸奶茶很棒...
🔄 嘗試分析（第 1 次）...
✅ 結構化分析成功
  ✅ 完成：['50嵐']
  📝 分析文本 3/6: 迷客夏的芝芝系列推薦...
🔄 嘗試分析（第 1 次）...
✅ 結構化分析成功
  ✅ 完成：['迷客夏']
  ⏸️ 批次完成，等待 1.0 秒...

📦 處理批次 2/2
  📝 分析文本 4/6: 清心福全的檸檬綠茶...
🔄 嘗試分析（第 1 次）...
✅ 結構化分析成功
  ✅ 完成：['清心福全']
  📝 分析文本 5/6: 今天天氣很好但沒喝飲料...
🔄 嘗試分析（第 1 次）...
✅ 結構化分析成功
  ✅ 完成：[]
  📝 分析文本 6/6: 麥吉的氣泡飲料很特別...
🔄 嘗試分析（第 1 次）...
✅ 結構化分析成功
  ✅ 完成：['麥吉']

🎉 批量分析完成！成功處理 6 個文本

📊 批量分析結果統計：
🏪 總共識別出 5 個不同品牌
📈 品牌統計：{'CoCo': 1, '50嵐': 1, '迷客夏': 1, '清心福全': 1, '麥吉': 1}
✅ 成功率：83.3%
